# Genetic Algorithms
Code for [Genetic Algorithms - Learn Python for Data Science #6](https://youtu.be/dSofAXnnFrY) <br>
Importing libraries.

In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

### Data
The data are MC generated (see below) to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere. This Cherenkov radiation (of visible to UV wavelengths) leaks through the atmosphere and gets recorded in the detector, allowing reconstruction of the shower parameters. The available information consists of pulses left by the incoming Cherenkov photons on the photomultiplier tubes, arranged in a plane, the camera. Depending on the energy of the primary gamma, a total of few hundreds to some 10000 Cherenkov photons get collected, in patterns (called the shower image), allowing to discriminate statistically those caused by primary gammas (signal) from the images of hadronic showers initiated by cosmic rays in the upper atmosphere (background). <br>

Typically, the image of a shower after some pre-processing is an elongated cluster. Its long axis is oriented towards the camera center if the shower axis is parallel to the telescope's optical axis, i.e. if the telescope axis is directed towards a point source. A principal component analysis is performed in the camera plane, which results in a correlation axis and defines an ellipse. If the depositions were distributed as a bivariate Gaussian, this would be an equidensity ellipse. The characteristic parameters of this ellipse (often called Hillas parameters) are among the image parameters that can be used for discrimination. The energy depositions are typically asymmetric along the major axis, and this asymmetry can also be used in discrimination. There are, in addition, further discriminating characteristics, like the extent of the cluster in the image plane, or the total sum of depositions.<br>

Attribute Information:

1. fLength: continuous # major axis of ellipse [mm]
2. fWidth: continuous # minor axis of ellipse [mm]
3. fSize: continuous # 10-log of sum of content of all pixels [in #phot]
4. fConc: continuous # ratio of sum of two highest pixels over fSize [ratio]
5. fConc1: continuous # ratio of highest pixel over fSize [ratio]
6. fAsym: continuous # distance from highest pixel to center, projected onto major axis [mm]
7. fM3Long: continuous # 3rd root of third moment along major axis [mm]
8. fM3Trans: continuous # 3rd root of third moment along minor axis [mm]
9. fAlpha: continuous # angle of major axis with vector to origin [deg]
10. fDist: continuous # distance from origin to center of ellipse [mm]
11. class: g,h # gamma (signal), hadron (background)
<br><br>
g = gamma (signal): 12332 <br>
h = hadron (background): 6688

For technical reasons, the number of h events is underestimated. In the real data, the h class represents the majority of the events.<br>

The simple classification accuracy is not meaningful for this data, since classifying a background event as signal is worse than classifying a signal event as background. For comparison of different classifiers an ROC curve has to be used. The relevant points on this curve are those, where the probability of accepting a background event as signal is below one of the following thresholds: 0.01, 0.02, 0.05, 0.1, 0.2 depending on the required quality of the sample of the accepted events for different experiments.



In [2]:
# load the data
telescope = pd.read_csv('data/MAGIC Gamma Telescope Data.csv')

# clean the data
telescope_shuffle = telescope.iloc[np.random.permutation(len(telescope))]
tele = telescope_shuffle.reset_index(drop=True)

# store 2 classes
tele['Class'] = tele['Class'].map({'g':0, 'h':1})
tele_class = tele['Class'].values

# first 5 elements
tele.head(5)

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,63.8128,26.2233,3.5143,0.1958,0.1088,44.5465,46.5522,18.3607,1.9190,284.6520,0
1,33.9347,10.8862,2.5328,0.4047,0.2155,-25.0369,-23.1659,3.4920,0.3340,132.9070,0
2,23.4914,10.2250,2.3181,0.5433,0.3005,32.2449,-16.3290,-9.4179,60.5491,26.0889,0
3,126.6650,50.7670,3.4115,0.1400,0.0766,-74.1327,-59.6968,-31.9915,14.2270,296.4660,0
4,25.6568,11.7431,2.5944,0.4249,0.2150,34.2181,17.2466,-5.1048,26.3933,219.5660,0


Preparing data, spliting into training, testing and validation sets.

In [3]:
training_indices, testing_indices = train_test_split(tele.index, 
                                                     stratify=tele_class, train_size=0.75, test_size=0.25)
# testing and validation sets are the same
validation_indices = testing_indices

### Genetic programming
Let Genetic Programming find best ML model and hyperparameters.

In [4]:
tpot = TPOTClassifier(generations=5, verbosity=2)
tpot.fit(tele.drop('Class', axis=1).loc[training_indices].values, 
         tele.loc[training_indices, 'Class'].values)

Generation 1 - Current best internal CV score: 0.8807571409227511
Generation 2 - Current best internal CV score: 0.882158829821941
Generation 3 - Current best internal CV score: 0.882158829821941
Generation 4 - Current best internal CV score: 0.882158829821941
Generation 5 - Current best internal CV score: 0.8833514917918197

Best pipeline: GradientBoostingClassifier(PolynomialFeatures(MinMaxScaler(input_matrix), degree=2, include_bias=False, interaction_only=False), learning_rate=0.1, max_depth=5, max_features=0.7000000000000001, min_samples_leaf=15, min_samples_split=18, n_estimators=100, subsample=0.55)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
        disable_update_check=False, early_stop=None, generations=5,
        max_eval_time_mins=5, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=100,
        random_state=None, scoring=None, subsample=1.0, use_dask=False,
        verbosity=2, warm_start=False)

### Accuracy
Scoring the accuracy.

In [6]:
tpot.score(tele.drop('Class', axis=1).loc[validation_indices].values,
           tele.loc[validation_indices, 'Class'].values)

0.8767613038906414

As can be seen, the best accuracy is 87.68%. <br>
Finally, we export the generated code in pipeline.py

In [7]:
tpot.export('data/pipeline.py')

True